In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()
print('WorkSpace Name: '+ ws.name,
    'Azure Region: ' + ws.location,
    'Subscription: '+ ws.subscription_id,
    'Resoure Group: ' + ws.resource_group, sep='\n')

WorkSpace Name: labuser55ml
Azure Region: eastus
Subscription: 3f43e823-597d-46de-8acc-3385c3f14d12
Resoure Group: rg55


In [2]:
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name='diabetes-experiment')

In [3]:
experiment

Name,Workspace,Report Page,Docs Page
diabetes-experiment,labuser55ml,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna()
y_df = x_df.pop('Y')

X_train, X_test, y_train, y_test = train_test_split(
                    x_df,
                    y_df,
                    test_size = 0.2,
                    random_state = 42
)

In [5]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import math
import numpy as np

alphas = [round(num, 1) for num in np.arange(0.1, 1.1, 0.1)]

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
for alpha in alphas:
    run = experiment.start_logging()
    run.log('alpha_value', alpha)

    model = Ridge(alpha=alpha)
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X=X_test)
    rmse = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))
    run.log('rmse', rmse)

    model_name = 'model_alpha_' + str(alpha) + '.pkl'
    file_name = 'outputs/' + model_name
    
    joblib.dump(value=model, filename=file_name) # pickle 파일 저장
    run.upload_file(name=model_name, path_or_stream=file_name) # Azure ML Studio에 업로드
    run.complete() # log 기록을 끝냄

    print(f'{alpha} experiment completed!')

0.1 experiment completed!
0.2 experiment completed!
0.3 experiment completed!
0.4 experiment completed!
0.5 experiment completed!
0.6 experiment completed!
0.7 experiment completed!
0.8 experiment completed!
0.9 experiment completed!
1.0 experiment completed!


# 베스트 모델 선택

In [7]:
minimum_rmse_runid = None
minimum_rmse = None

for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
    
    # print('RMSE = ' + str(run_metrics['rmse']) + ' id = ' + run_details['runId'])
    
    run_rmse = run_metrics['rmse']
    run_id = run_details['runId']

    if minimum_rmse is None: # 첫번째 rmse 기록
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse: # 더 낮은 RMSE를 찾을 시에 교체
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id

print('Best run-id: ' + minimum_rmse_runid)
print('Best RMSE: ' + str(minimum_rmse))

Best run-id: bc221134-c93e-42e3-9b5a-e535d98ebc1b
Best RMSE: 53.85527385056211


In [8]:
from azureml.core import Run
best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)
print(best_run.get_file_names())
best_run.download_file(name=str(best_run.get_file_names()[0])) # Best Model Download

['model_alpha_0.1.pkl', 'outputs/model_alpha_0.1.pkl']


In [11]:
import numpy as np
from azureml.core import Dataset

np.savetxt('features.csv', X_train, delimiter=',')
np.savetxt('labels.csv', y_train, delimiter=',')

datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv','./labels.csv'],
                        target_path='diabetes-experiment',
                        overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(
                    path=[(datastore,'diabetes-experiment/features.csv')]
)
output_dataset = Dataset.Tabular.from_delimited_files(
                    path=[(datastore,'diabetes-experiment/labels.csv')]
)

Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [15]:
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

 # 베스트 모델을 Azure ML Studio에 등록
model = Model.register(workspace=ws, 
                        model_name='diabetes-experiment-model',
                        model_path=f'./{str(best_run.get_file_names()[0])}',
                        model_framework=Model.Framework.SCIKITLEARN,
                        model_framework_version=sklearn.__version__,
                        sample_input_dataset=input_dataset,
                        sample_output_dataset=output_dataset,
                        resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                        description='Ridge regression model to predict diabetes progression',
                        tags={'area':'diabetes', 'type':'regresion'}
)

print('Name: ', model.name)
print('Version: ', model.version)

Registering model diabetes-experiment-model
Name:  diabetes-experiment-model
Version:  1


In [16]:
# 모델 배포
server_name = 'diabetes-service'

service = Model.deploy(ws, server_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True) # 중간 결과물 확인

/tmp/ipykernel_158968/2253787028.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, server_name, [model], overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-13 07:29:19+00:00 Creating Container Registry if not exists..
2023-02-13 07:39:19+00:00 Registering the environment..
2023-02-13 07:39:23+00:00 Uploading autogenerated assets for no-code-deployment..
2023-02-13 07:39:31+00:00 Building image..
2023-02-13 07:49:51+00:00 Generating deployment configuration.
2023-02-13 07:49:52+00:00 Submitting deployment to compute..
2023-02-13 07:50:01+00:00 Checking the status of deployment diabetes-service..
2023-02-13 07:51:38+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
import json

input_pay_load = json.dumps({'data': X_train[0:2].values.tolist(),
 'method': 'predict'})

output = service.run(input_pay_load) # 서비스 실행
print(output)

{'predict': [184.66566335446362, 151.8625020138362]}
